In [2]:
# import pandas and wbdata
import pandas as pd
import wbdata as wb

In [22]:
# Read the CSVs into dataframes
df_a = pd.read_csv('Resources/inpe_brazilian_amazon_fires_1999_2019.csv')
df_b = pd.read_csv('Resources/def_area_2004_2019.csv')

# Create a mapping dictionary
state_mapping = {
    'AC': 'ACRE',
    'AM': 'AMAZONAS',
    'AP': 'AMAPA',
    'MA': 'MARANHAO',
    'MT': 'MATO GROSSO',
    'PA': 'PARA',
    'RO': 'RONDONIA',
    'RR': 'RORAIMA',
    'TO': 'TOCANTINS'
}

# Map the full state names in df_a to their initials
df_a['state_initials'] = df_a['state'].map({v: k for k, v in state_mapping.items()})

# Convert the 'year' column in both dataframes to the same type
df_a['year'] = df_a['year'].astype(str)
df_b['Ano/Estados'] = df_b['Ano/Estados'].astype(str)

# Melt df_b to long format to facilitate the merge
df_b_melted = pd.melt(df_b, id_vars=['Ano/Estados'], value_vars=state_mapping.keys(), 
                      var_name='state_initials', value_name='deforestation_area')

# Merge the two dataframes on the year and state initials
merged_df = pd.merge(df_a, df_b_melted, left_on=['year', 'state_initials'], right_on=['Ano/Estados', 'state_initials'], how='outer')

# Filter to keep only data from 2004 onwards
merged_df = merged_df[merged_df['year'] > '2004']

# Drop the state initials column and the duplicate 'Ano/Estados' column
merged_df.drop(['state_initials', 'Ano/Estados'], axis=1, inplace=True)

merged_df


,year,month,state,latitude,longitude,firespots,deforestation_area
559,2005,1,ACRE,-8.065450,-72.086500,20,592.0
560,2005,3,ACRE,-8.780600,-70.126200,5,592.0
561,2005,4,ACRE,-10.807000,-69.092000,2,592.0
562,2005,5,ACRE,-9.607824,-69.935294,17,592.0
563,2005,6,ACRE,-9.504400,-69.515640,75,592.0
...,...,...,...,...,...,...,...
2099,2019,8,TOCANTINS,-7.555371,-48.651968,62,21.0
2100,2019,9,TOCANTINS,-7.062450,-48.643700,40,21.0
2101,2019,10,TOCANTINS,-7.334791,-48.603163,43,21.0
2102,2019,11,TOCANTINS,-6.322091,-48.443023,44,21.0
